<a href="https://colab.research.google.com/github/LEEGWONSEOK/MLDL_Basic/blob/main/5_2_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
## data 분리

from sklearn.model_selection import train_test_split

# data에서 train, test 셋으로 분리
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# train set 에서 train, validation으로 다시 분리
sub_input, valid_input, sub_target, valid_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# train, valid, test 비율 확인! >> 잘 뽑힌걸 확인할 수 있다
print(sub_input.shape, valid_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [ ]:
## 결정트리모델 생성

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)    # 모델 객체 생성
dt.fit(sub_input, sub_target)

# train, valid로 우선 점수 비교해본다!
print(dt.score(sub_input, sub_target))
print(dt.score(valid_input, valid_target))

# 대충 과적합!

0.9971133028626413
0.864423076923077


In [ ]:
## 교차검증
# 교차검증을 하는 이유? : 무작위로 여러번 모델을 돌려서 과적합을 줄이기 위해!

from sklearn.model_selection import cross_validate  # 교차검증 모듈

scores = cross_validate(dt, train_input, train_target)
print(scores)

# cross_validate 의 출력값은 fit_time, score_time, test_score 

{'fit_time': array([0.00869203, 0.00680685, 0.0071156 , 0.00702786, 0.00652313]), 'score_time': array([0.0006845 , 0.00055385, 0.00053191, 0.00059223, 0.00057864]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

# 교차검증은 나온 점수들의 평균 값으로 확인가능하다!
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
## 교차검증 : K폴드 모델

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())  # 이거는 따로 객체생성을 안하는 건지 아니면 형태가 다른지
print(np.mean(scores['test_score']))

# 위와 점수가 동일한 것을 확인할 수 있다

0.855300214703487


In [ ]:
## 하이퍼파라미터 튜닝 : 그리드서치

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}  # parameter를 딕셔너리 형태로 작업한다!

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs : CPU 사용 선택 파라미터
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [ ]:
dt = gs.best_estimator_   # 나온 값들 중에 가장 높은 점수를 채택
print(dt.score(train_input, train_target))
print(gs.best_params_)    # 어떤 파라미터값이 가장 높은지 확인

# min_impurity_decrease = 0.0001 일때 점수가 가장 높다는 것 확인!

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])  # 돌린 점수 전부 확인

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [ ]:
# 파라미터 좀더 넓혀서 해보기
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs2 = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs2.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [ ]:
print(gs2.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))  # 여기서는 다양하게 나온 값들 중에 가장 높은 값으로 선택!(max)

# 그리드서치를 통해 점수가 조금 오른 것을 확인할 수 있다

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8681929740134745


In [ ]:
## 랜덤 서치
# 그리드 서치의 단점은 일일이 값을 다 돌려보기때문에 오래걸린다, 랜덤서치를 통해 조금 효율적인 튜닝이 가능

from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

# 10번 밖에 안돌려서 점수가 뒤죽박죽처럼 보인다

array([6, 4, 4, 8, 3, 4, 5, 5, 9, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

# 하지만 1000번이나 돌리면 어느정도 순서가 맞다!

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101,  92, 100, 104,  97,  98, 116,  92, 101,  99]))

In [39]:
from sklearn.model_selection import RandomizedSearchCV    # 랜덤서치 모듈

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [41]:
print(rs.best_params_)

print(np.max(rs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [42]:
dt_rs = rs.best_estimator_

print(dt_rs.score(test_input, test_target))

# 좀더 과적합이 줄어들었다는 것을 알 수 있다

0.86
